# Get t events
Convert indicator to t events

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [5]:
df = pd.read_csv("../data/BTCUSDT-volume-bars.csv", index_col=0, parse_dates=True)
ind = pd.read_csv("../data/indicator.csv", index_col=0, parse_dates=True, squeeze=True)

In [14]:
df = df[-500_000:]

In [12]:
def get_t_events(g_raw, h, m):
    """
    Implementation of the symmetric CUSUM filter seen in chapter 2.5.2
    
    Arguments:
    g_raw -- the raw time series to filter (possibly pandas series)
    h -- threshold pandas series vol
    m -- minimum threshold (float, ex: 0.02)
    
    Purpose:
    The CUSUM filter is a quality-control method, designed to detect a shift in the mean value of a measured quantity away from a target value. 
    """
    h = h.map(lambda x: x if x>=m else m)
    t_events, s_pos, s_neg = [], 0, 0
    diff = np.log(g_raw).diff().dropna()
    for i in diff.index[1:]:
        try:
            pos, neg = float(s_pos+diff.loc[i]), float(s_neg+diff.loc[i])
        except Exception as e:
            print(e)
            print(s_pos+diff.loc[i], type(s_pos+diff.loc[i]))
            print(s_neg+diff.loc[i], type(s_neg+diff.loc[i]))
            break
        s_pos, s_neg=max(0., pos), min(0., neg)
        th = h[h.index.get_loc(i, method="pad")]
        if s_neg<-th:
            s_neg=0;t_events.append(i)
#         elif s_pos>th:
#             s_pos=0;t_events.append(i)
    return pd.DatetimeIndex(t_events)

In [13]:
print(f"max: {max(ind[1:].dropna())}")
print(f"min: {min(ind[1:].dropna())}")
print(f"mean: {np.mean(ind)}")
print(f"median: {np.median(ind[1:].dropna())}")

max: 0.0430782553456884
min: 0.0028498335389419
mean: 0.020184044125735667
median: 0.0201898914119819


In [ ]:
t_events = get_t_events(df.close,h=ind, m=0.02)
t_events

In [ ]:
side = df.close.reindex(t_events).diff().shift(-1).map(lambda x: 1 if x>0 else 0)
side

In [ ]:
side.to_csv("../data/side.csv")

## Plotting

In [ ]:
a = 700000
b = 800000
test = df[a:b].reindex(t_events)
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(df.close[a:b], color="gray", alpha=.5)
ax.scatter(test.index, test.close)